## Filtering and Annotation Tutorial

### Filter

You can filter the rows of a table with [Table.filter](https://hail.is/docs/0.2/hail.Table.html#hail.Table.filter).   This returns a table of those rows for which the expression evaluates to `True`.

In [1]:
import hail as hl

hl.utils.get_movie_lens('data/')
users = hl.read_table('data/users.ht')

Loading BokehJS ...

Initializing Hail with default parameters...


Running on Apache Spark version 3.5.0
SparkUI available at http://hostname-e7df4bc790:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.134-952ae203dbbe
LOGGING: writing to /io/hail/python/hail/docs/tutorials/hail-20250307-1211-0.2.134-952ae203dbbe.log


2025-03-07 12:11:58.264 Hail: INFO: Movie Lens files found!


In [2]:
users.filter(users.occupation == 'programmer').count()

66

We can also express this query in multiple ways using [aggregations](https://hail.is/docs/0.2/tutorials/04-aggregation.html): 

In [3]:
users.aggregate(hl.agg.filter(users.occupation == 'programmer', hl.agg.count()))

66

In [4]:
users.aggregate(hl.agg.counter(users.occupation == 'programmer'))[True]

66

### Annotate

You can add new fields to a table with [annotate](https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate).  As an example, let's create a new column called `cleaned_occupation` that replaces missing entries in the `occupation` field labeled as 'other' with 'none.'

In [5]:
missing_occupations = hl.set(['other', 'none'])

t = users.annotate(
    cleaned_occupation = hl.if_else(missing_occupations.contains(users.occupation),
                                    hl.missing('str'),
                                    users.occupation))
t.show()

+-------+-------+-----+-----------------+---------+--------------------+
|    id |   age | sex | occupation      | zipcode | cleaned_occupation |
+-------+-------+-----+-----------------+---------+--------------------+
| int32 | int32 | str | str             | str     | str                |
+-------+-------+-----+-----------------+---------+--------------------+
|     1 |    24 | "M" | "technician"    | "85711" | "technician"       |
|     2 |    53 | "F" | "other"         | "94043" | NA                 |
|     3 |    23 | "M" | "writer"        | "32067" | "writer"           |
|     4 |    24 | "M" | "technician"    | "43537" | "technician"       |
|     5 |    33 | "F" | "other"         | "15213" | NA                 |
|     6 |    42 | "M" | "executive"     | "98101" | "executive"        |
|     7 |    57 | "M" | "administrator" | "91344" | "administrator"    |
|     8 |    36 | "M" | "administrator" | "05201" | "administrator"    |
|     9 |    29 | "M" | "student"       | "01002" | "student"          |
|    10 |    53 | "M" | "lawyer"        | "90703" | "lawyer"           |
+-------+-------+-----+-----------------+---------+--------------------+
showing top 10 rows

Compare this to what we had before:

In [6]:
users.show()

+-------+-------+-----+-----------------+---------+
|    id |   age | sex | occupation      | zipcode |
+-------+-------+-----+-----------------+---------+
| int32 | int32 | str | str             | str     |
+-------+-------+-----+-----------------+---------+
|     1 |    24 | "M" | "technician"    | "85711" |
|     2 |    53 | "F" | "other"         | "94043" |
|     3 |    23 | "M" | "writer"        | "32067" |
|     4 |    24 | "M" | "technician"    | "43537" |
|     5 |    33 | "F" | "other"         | "15213" |
|     6 |    42 | "M" | "executive"     | "98101" |
|     7 |    57 | "M" | "administrator" | "91344" |
|     8 |    36 | "M" | "administrator" | "05201" |
|     9 |    29 | "M" | "student"       | "01002" |
|    10 |    53 | "M" | "lawyer"        | "90703" |
+-------+-------+-----+-----------------+---------+
showing top 10 rows

Note: `annotate` is functional: it doesn't mutate `users`, but returns a new table.  This is also true of `filter`.  In fact, all operations in Hail are functional.

In [7]:
users.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'id': int32 
    'age': int32 
    'sex': str 
    'occupation': str 
    'zipcode': str 
----------------------------------------
Key: ['id']
----------------------------------------


### Select and Transmute
There are two other annotate methods: [select](https://hail.is/docs/0.2/hail.Table.html#hail.Table.select) and [transmute](https://hail.is/docs/0.2/hail.Table.html#hail.Table.transmute).  `select` allows you to create new tables from old ones by selecting existing fields, or creating new ones.

First, let's extract the `sex` and `occupation` fields:

In [8]:
users.select(users.sex, users.occupation).show()

+-------+-----+-----------------+
|    id | sex | occupation      |
+-------+-----+-----------------+
| int32 | str | str             |
+-------+-----+-----------------+
|     1 | "M" | "technician"    |
|     2 | "F" | "other"         |
|     3 | "M" | "writer"        |
|     4 | "M" | "technician"    |
|     5 | "F" | "other"         |
|     6 | "M" | "executive"     |
|     7 | "M" | "administrator" |
|     8 | "M" | "administrator" |
|     9 | "M" | "student"       |
|    10 | "M" | "lawyer"        |
+-------+-----+-----------------+
showing top 10 rows

We can also create a new field that stores the age relative to the average. Note that new fields *must* be assigned a name (in this case `mean_shifted_age`):

In [9]:
mean_age = round(users.aggregate(hl.agg.stats(users.age)).mean)
users.select(users.sex, users.occupation, mean_shifted_age = users.age - mean_age).show()

+-------+-----+-----------------+------------------+
|    id | sex | occupation      | mean_shifted_age |
+-------+-----+-----------------+------------------+
| int32 | str | str             |            int32 |
+-------+-----+-----------------+------------------+
|     1 | "M" | "technician"    |              -10 |
|     2 | "F" | "other"         |               19 |
|     3 | "M" | "writer"        |              -11 |
|     4 | "M" | "technician"    |              -10 |
|     5 | "F" | "other"         |               -1 |
|     6 | "M" | "executive"     |                8 |
|     7 | "M" | "administrator" |               23 |
|     8 | "M" | "administrator" |                2 |
|     9 | "M" | "student"       |               -5 |
|    10 | "M" | "lawyer"        |               19 |
+-------+-----+-----------------+------------------+
showing top 10 rows

`transmute` replaces any fields mentioned on the right-hand side with the new fields, but leaves unmentioned fields unchanged.  `transmute` is useful for transforming data into a new form. Compare the following two snippts of code. The second is identical to the first with `transmute` replacing `select.`

In [10]:
missing_occupations = hl.set(['other', 'none'])

t = users.select(
    cleaned_occupation = hl.if_else(missing_occupations.contains(users.occupation),
                                    hl.missing('str'),
                                    users.occupation))
t.show()

+-------+--------------------+
|    id | cleaned_occupation |
+-------+--------------------+
| int32 | str                |
+-------+--------------------+
|     1 | "technician"       |
|     2 | NA                 |
|     3 | "writer"           |
|     4 | "technician"       |
|     5 | NA                 |
|     6 | "executive"        |
|     7 | "administrator"    |
|     8 | "administrator"    |
|     9 | "student"          |
|    10 | "lawyer"           |
+-------+--------------------+
showing top 10 rows

In [11]:
missing_occupations = hl.set(['other', 'none'])

t = users.transmute(
    cleaned_occupation = hl.if_else(missing_occupations.contains(users.occupation),
                                    hl.missing('str'),
                                    users.occupation))
t.show()

+-------+-------+-----+---------+--------------------+
|    id |   age | sex | zipcode | cleaned_occupation |
+-------+-------+-----+---------+--------------------+
| int32 | int32 | str | str     | str                |
+-------+-------+-----+---------+--------------------+
|     1 |    24 | "M" | "85711" | "technician"       |
|     2 |    53 | "F" | "94043" | NA                 |
|     3 |    23 | "M" | "32067" | "writer"           |
|     4 |    24 | "M" | "43537" | "technician"       |
|     5 |    33 | "F" | "15213" | NA                 |
|     6 |    42 | "M" | "98101" | "executive"        |
|     7 |    57 | "M" | "91344" | "administrator"    |
|     8 |    36 | "M" | "05201" | "administrator"    |
|     9 |    29 | "M" | "01002" | "student"          |
|    10 |    53 | "M" | "90703" | "lawyer"           |
+-------+-------+-----+---------+--------------------+
showing top 10 rows

### Global Fields

Finally, you can add global fields with [annotate_globals](https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate_globals).  Globals are useful for storing metadata about a dataset or storing small data structures like sets and maps.

In [12]:
t = users.annotate_globals(cohort = 5, cloudable = hl.set(['sample1', 'sample10', 'sample15']))
t.describe()

----------------------------------------
Global fields:
    'cohort': int32 
    'cloudable': set<str> 
----------------------------------------
Row fields:
    'id': int32 
    'age': int32 
    'sex': str 
    'occupation': str 
    'zipcode': str 
----------------------------------------
Key: ['id']
----------------------------------------


In [13]:
t.cloudable

<SetExpression of type set<str>>

In [14]:
hl.eval(t.cloudable)

{'sample1', 'sample10', 'sample15'}

### Exercises


- [Z-score normalize](https://en.wikipedia.org/wiki/Standard_score) the age field of `users`.
- Convert `zip` to an integer.  Hint: Not all zipcodes are US zipcodes!  Use [hl.int32](https://hail.is/docs/0.2/functions/constructors.html#hail.expr.functions.int32) to convert a string to an integer.  Use [StringExpression.matches](https://hail.is/docs/0.2/expressions.html#hail.expr.expressions.StringExpression.matches) to see if a string matches a regular expression.